## The upper bound of CV accuracy of any ML models on the tested dataset (initial sampling + 5 AL runs) was calculated as one minus intrinsic error of the dataset. The intrinsic error was estimated as one minus the average CV accuracy of multiple deep neural networks that overfit the dataset.

In [ ]:
import numpy as np
from numpy.random import random
from numpy import vstack, hstack
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.neural_network import MLPClassifier
from Data.datasets import save_obj, load_obj, data_preprocess
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

In [ ]:
# Import datasets (2 initial sampling and 5 active learning)
df_std = pd.read_csv('Data/005.morph phase mapping.csv')
df_std.index = list(df_std['index'])
df_score = df_std.filter(['score'], axis = 1)
df_std = df_std.drop(['index', 'score'], axis = 1)

iterations = [48, 72, 96, 120, 144, 168]

In [ ]:
# Generate a list of number of hidden units
hidden_unit = np.logspace(start = 0.5, stop = 2, num = 20)
hidden_unit = list(set([int(x) for x in hidden_unit]))
hidden_unit.sort()

In [ ]:
# Use leave one out to calculate average error, and monitor it with increaseing number of hidden units.
error_matrix = []
error_std_matrix = []
import statistics

for iteration in tqdm(iterations):
    
    error_list = []
    error_std_list = []
    
    df_tested = df_std[:iteration]
    df_score_tested = df_score[:iteration]
    
    for num_unit in hidden_unit:
        MLP = MLPClassifier(hidden_layer_sizes = (num_unit,), solver='lbfgs', activation = 'relu')
        error = []
        for test in df_tested.index:
            MLP.fit(np.array(df_tested.drop([test], axis = 'index')), \
                    np.array(df_score_tested.drop([test], axis = 'index')).reshape(len(df_score_tested.drop([test], axis = 'index')),))
            error.append(1 - MLP.score(np.array(df_tested.filter([test], axis = 'index')), \
                                       np.array(df_score_tested.filter([test], axis = 'index')).reshape(1,)))
            
        error_list.append(sum(error)/len(error))
        error_std_list.append(statistics.stdev(error))
        
    error_matrix.append(error_list)
    error_std_matrix.append(error_std_list)

In [ ]:
MLP

In [ ]:
error_matrix = np.array(error_matrix)
error_std_matrix = np.array(error_std_matrix)

df_error = pd.DataFrame(columns = hidden_unit, data = error_matrix)
df_error_std = pd.DataFrame(columns = hidden_unit, data = error_std_matrix)

In [ ]:
df_error

In [ ]:
%matplotlib notebook
color_list = ['black', 'purple', 'blue', 'green', 'red', 'orange']

for index in df_error.index:
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot()
    
    ax.plot(np.log10(hidden_unit), list(df_error.loc[index]*100), c = color_list[index], linewidth = 1, linestyle = 'dashed')
    ax.scatter(np.log10(hidden_unit), list(df_error.loc[index]*100), s = 100, c = color_list[index])
    
#     ax.plot(np.log10(hidden_unit), [17]*19, c = 'gray', linewidth = 1, linestyle = 'dashed')
    
    ax.set_title('iteration'+ str(index))
    ax.set_xlim(0.4,2.1)
    ax.set_ylim(10,40)
    
    ax.set_xticks(np.arange(0.5,2.5,0.5))
    ax.set_yticks(np.arange(10,45,5))
    ax.tick_params(axis="x", labelsize=15)
    ax.tick_params(axis="y", labelsize=15)
    # plt.xlabel('log(Hidden units)')
    # plt.ylabel('Error')
    plt.savefig('Graphs_3/intrinsic error_iteration'+ str(index) +'_2ndversion.svg', format = 'svg', transparent = 'True')

## Calculate the lower bound of prediciton accuracy

In [ ]:
p1 = (list(df_score.score).count(1))/168
p2 = (list(df_score.score).count(3))/168
p3 = (list(df_score.score).count(4))/168
gini_imp = p1*(1-p1) + p2*(1-p2) + p3*(1-p3)

In [ ]:
1- gini_imp

In [ ]:
p1

In [ ]:
p2

In [ ]:
p3

In [ ]:
gini_imp